In [ ]:
from modules import *
from functions import *

In [ ]:
N = int(500)                                                        # Number of neurons
a = 10                                                              # Parameters for propensity function
theta_stim = 90                                                     # Angle to stimulate at 
n_test_angles = 100                                                 # Number of angles to use to test for preferred orientation
vars = np.random.lognormal(2, 0.6, N)                               # Width of each neuron's tuning curve
learning_rate = 1e-2                                                # Learning rate

n_days = 28                                                         # Number of days to run for
n_norm_per_day = 1                                                  # How many times to normalise the weights per day
n_steps_per_norm = 30                                               # How often to normalise the weights
n_steps = n_steps_per_norm * n_norm_per_day * n_days                # Number of steps to run for
init_steps = 300                                                    # Number of trials to run to settle to a baseline

hebb_scaling = 0.3                                                  # Scaling of Hebbian component
rand_scaling = 1                                                    # Scaling of random component 

# Initialise

W_init = initialise_W(N, vars)                                                                                                      # Initialise weights 
W_baseline = prerun(W_init, theta_stim, a, hebb_scaling, rand_scaling, learning_rate, n_steps_per_norm, init_steps)                 # Settle to a baseline

POs = []; ratios = []
W = np.zeros((N, N, n_steps+1)); W[:, :, 0] = W_baseline; W_per_day = np.zeros((N, N, n_days))

# Run trials
for t in tqdm(range(n_steps)):
    W_old = W[:, :, t]
    H = single_hebbian_component(N, W_old, theta_stim, type='baseline')                          # Hebbian component - outer product of pre- and post-synaptic activity
    eta = np.random.randn(N, N)                                                                  # Random component - sample from normal distribution
    prop_function = propensity(W_old, a)                                                         # Propensity function - tanh(w)
    hebb =  hebb_scaling * H                                                                     # Scaled Hebbian component 
    rand =  rand_scaling * eta                                                                   # Scaled random component
    W_new = W_old + learning_rate * prop_function * (hebb + rand)                                # Update weights

    if t % n_steps_per_norm == 0:                                                                # Perform normalisation after every N_theta steps 
        normalisation(W_new)
        if t % (n_steps_per_norm * n_norm_per_day) == 0:                                         # Save weights and preferred orientations every day 
            W_per_day[:, :, t // (n_steps_per_norm * n_norm_per_day)] = W_new                    

    W[:, :, t+1] = W_new

## Supp. Fig. 3a

In [ ]:
def supp_fig_3a(W_baseline):
    fig, ax = plt.subplots(figsize=(3, 3), dpi=180)
    im = ax.imshow(W_baseline, vmin=0, vmax=0.04)
    cbar = plt.colorbar(im, ax=ax, fraction=0.045, pad=0.04)
    cbar.ax.set_yticks([0, 0.02, 0.04])
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_yticks([0, 500])
    ax.set_xticks([0, 500])
    ax.set_xlabel('post', labelpad=0.5, fontstyle='italic', c='gray')
    ax.set_ylabel('pre', labelpad=0.5, fontstyle='italic', c='gray')
    fig.show()


## Supp. Fig. 3c

In [ ]:
def supp_fig_3c(W):
    fig, ax = plt.subplots(figsize=(2.5, 2.5), dpi=180)
    ax2 = ax.twinx()
    x = np.linspace(0, 1, 100)
    ax.plot(x, propensity(x, 10), c='firebrick')
    ax2.hist(W[:, :, -1].flatten(), bins=103, density=False, color='gray', alpha=0.5)
    ax2.set_yscale('log')
    ax.set_xlim([0, 0.4]); ax.set_xticks([0, 0.2, 0.4])
    ax.set_ylim([0, 1.01]); ax.set_yticks([0, 0.5, 1])
    ax.set_xlabel(r"Weight, $w$")
    ax.set_ylabel(r"$\rho(w)$", rotation=0, labelpad=20)
    ax2.set_ylabel(r"Count", rotation=270, labelpad=20, color='darkgray')
    ax2.spines['right'].set_color('darkgray'); ax2.tick_params(axis='y', colors='darkgray')
    sns.despine(right=False)
    fig.show()


## Supp. Fig. 3b

In [ ]:
def supp_fig_3b():
    N=100000
    vars = np.random.lognormal(2, 0.6, N)
    fig, ax = plt.subplots(figsize=(2.3, 2.3), dpi=180)
    ax.hist(vars, bins=100, density=True, histtype='step', lw=1, color='k')
    ax.hist(vars, bins=100, density=True, histtype='stepfilled', color='firebrick')
    ax.set_yticks([0, 0.1]); ax.set_xlim(-4, 100)
    ax.set_xlabel(r'Tuning curve width $ \; [\degree]$')
    ax.set_ylabel('Frequency')
    fig.show()

## Fig. 3b

In [ ]:
def fig_3b():
    x = np.linspace(0, 0.2)
    y = propensity(x, a=20)
    fig, ax = plt.subplots(1, 1, figsize=(1.2, 1.2), dpi=140)
    ax.plot(x, y, c='k', lw=1.5)
    ax.set_xlabel(r'w', labelpad=5)
    ax.set_ylabel(r'$\rho(w)$', labelpad=1)
    ax.set_ylim(0, 1.01); ax.set_xlim(0, 0.2); ax.set_xticks([]); ax.set_yticks([0, 1])
    fig.show()